In this notebook we will deal with multiple destinations and drivers. Based on the given number of drivers we use K-means to cluster the destinations.  Then, within each driver's cluster, we create an algorithm that uses the A* algorithm to compute the best ordering of destinations, along with the optimal path from place to place.  

We use edge weights to allow for optimizing by travel time or total distance travelled. 

As with our work with one destination we will allow for further customization by the user - avoiding highways or providing a list of streets they would like to avoid.

NOTE:  the result of this notebook will be outputs that show the route, edge by edge, along with edge information.  Destination markers and turn directions have been added.  Further work will be done to simplify this output as was done with the one destination.

In [1]:
import osmnx as ox
import networkx as nx
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from geopy.geocoders import Nominatim
import geopy.distance
from math import atan2, pi
from sklearn.cluster import KMeans

In [2]:
# Build a graph - this is the Whitby/Oshawa area for testing purposes
G = ox.graph_from_bbox(43.984503, 43.862879, -78.954552, -78.821660, network_type='drive')
# Add edge speed attributes
G = ox.add_edge_speeds(G)

In [3]:
# Check one edge 
G.edges[(699620, 4238293220, 0)]

{'osmid': 424417362,
 'lanes': '2',
 'name': 'Garrard Road',
 'highway': 'residential',
 'oneway': False,
 'length': 12.565,
 'speed_kph': 46.5}

In [4]:
# Check total number of edges 
len(G.edges)

12041

In [5]:
# We want to ensure every edge properly has a speed attribute 
good = 0
for edge in G.edges:
    if 'speed_kph' in G.edges[edge].keys():
        good += 1
    else:
        print(G.edges[edge]['highway'])
print(good)

12041


In [6]:
# We want to make sure all of the speeds are appropriate values
speeds = []
for edge in G.edges:
    speeds.append(G.edges[edge]['speed_kph'])

speeds = set(speeds)
print(speeds)

{64.4, 65.0, 100.0, 70.0, 71.0, 40.0, 75.0, 45.0, 46.5, 80.0, 49.6, 50.0, 20.0, 55.0, 55.2, 60.0, 30.0}


In [7]:
# Add travel time attributes
G = ox.add_edge_travel_times(G)

In [8]:
# Check one edge 
G.edges[(699620, 4238293220, 0)]

{'osmid': 424417362,
 'lanes': '2',
 'name': 'Garrard Road',
 'highway': 'residential',
 'oneway': False,
 'length': 12.565,
 'speed_kph': 46.5,
 'travel_time': 1.0}

In [9]:
# We want to ensure every edge properly has a travel time
good = 0
for edge in G.edges:
    if 'travel_time' in G.edges[edge].keys():
        good += 1
    else:
        print(G.edges[edge]['highway'])
print(good)

12041


In [10]:
# We want to make sure all of the travel times are appropriate values
times = []
for edge in G.edges:
    times.append(G.edges[edge]['travel_time'])

times = set(times)
print(min(times))
print(max(times))

0.3
143.6


In [11]:
def build_graph(east, west, south, north):
    """
    Define the limits of the map by specifying the boundary longitudes
    and latitudes.
    
    Create the graph object.
    
    Return the graph.
    """
    G = ox.graph_from_bbox(east, west, south, north, network_type = 'drive') 
    G = ox.add_edge_speeds(G)
    G = ox.add_edge_travel_times(G)
    return G

### Helper Functions

In [12]:
def address_to_coord(address):
    """
    Take address information and use Geopy to get its coordinates.   
    """
    locator = Nominatim(user_agent = 'capstone2_project')
    location = locator.geocode(address)
    
    if location:
        return (location.latitude, location.longitude)
    else:
        print('Address Not Found')

In [13]:
# Define starting point
start = address_to_coord('209 Dundas St E Whitby')
start

(43.879921100000004, -78.94055259302831)

In [14]:
# Define destinations
dest1 = '910 Dundas St W Whitby'
dest2 = '3100 Garden Street Whitby'
dest3 = '4081 Thickson Road N Whitby'
dest4 = '728 Anderson Street Whitby'
dest5 = '1801 Dundas St E Whitby'
dest6 = '3500 Brock St N Whitby'

destaddresses = [dest1, dest2, dest3, dest4, dest5, dest6]

dest1coords = address_to_coord(dest1)
dest2coords = address_to_coord(dest2)
dest3coords = address_to_coord(dest3)
dest4coords = address_to_coord(dest4)
dest5coords = address_to_coord(dest5)
dest6coords = address_to_coord(dest6)

In [15]:
destinations = [dest1coords, dest2coords, dest3coords, dest4coords, dest5coords, dest6coords]
destinations

[(43.8761031, -78.9642643),
 (43.90189732393074, -78.94123924628661),
 (43.9590881, -78.9442295),
 (43.8934292, -78.9266608),
 (43.8880942, -78.9052277),
 (43.91177930263158, -78.95641609868422)]

In [16]:
def cluster_destinations(destinations, drivers):
    """
    Use the list of destination coordinates and the number of drivers.
    Perform K-Means to cluster the destinations.
    Returns the list of destination tuples separated as sublists according
    to their cluster.
    """
    kmeans = KMeans(n_clusters=drivers)
    kmeans.fit(destinations)
    clusters = []
    for i in range(drivers):
        group = []
        for j in range(len(destinations)):
            if kmeans.labels_[j] == i:
                group.append(destinations[j])
        clusters.append(group)
    return clusters

In [17]:
clusters = cluster_destinations(destinations, 3)
clusters

[[(43.8934292, -78.9266608), (43.8880942, -78.9052277)],
 [(43.8761031, -78.9642643),
  (43.90189732393074, -78.94123924628661),
  (43.91177930263158, -78.95641609868422)],
 [(43.9590881, -78.9442295)]]

In [18]:
def coords_to_node(graph, coords):
    """
    Take coordinates and returns the closest node in the graph.
    """
    return ox.get_nearest_node(graph, coords)

In [19]:
start_node = coords_to_node(G, start)
start_node

392180675

In [20]:
def dest_address_and_node(graph, dest_addresses):
    """
    To go back and forth between the actual address and the graph node.
    This will be used in the route display sections.
    """
    dest_dict = dict()
    for address in dest_addresses:
        coords = address_to_coord(address)
        node = coords_to_node(graph, coords)
        dest_dict[address] = node
    return dest_dict

In [21]:
dest_dict = dest_address_and_node(G, destaddresses)
dest_dict

{'910 Dundas St W Whitby': 392166620,
 '3100 Garden Street Whitby': 392164099,
 '4081 Thickson Road N Whitby': 7157881216,
 '728 Anderson Street Whitby': 392171046,
 '1801 Dundas St E Whitby': 290584416,
 '3500 Brock St N Whitby': 392177756}

In [22]:
# Get destination nodes
# Keep as a list of lists - according to clusters
destination_nodes = []
for i in range(len(clusters)):
    inodes = []
    for x in clusters[i]:
        inodes.append(coords_to_node(G, x))
    destination_nodes.append(inodes)
destination_nodes

[[392171046, 290584416], [392166620, 392164099, 392177756], [7157881216]]

In [23]:
def node_to_coords(graph, node):
    """
    Gets the latitude and logitude of a graph node.
    """
    return (graph.nodes[node]['y'], graph.nodes[node]['x'])

In [24]:
def dist(source_node, destination_node):
    """
    Takes two nodes, gets their coordinates and uses Geopy to calculate 
    the distance between them in metres. 
    """
    return geopy.distance.distance(node_to_coords(G, source_node), 
                                    node_to_coords(G, destination_node)).m

In [25]:
# We now work within a cluster 
# We want to keep track of the destinations visited within that cluster 
# and the optimal path from to visit each destination within the cluster.
visited = []
route = []

In [26]:
def aStarMulti(start_node, destinations_nodes, graph, weight, visited, route):
    """
    This algorithm will determine the order in which to visit each destination in each cluster,
    and the optimal path from the start location, through all destinations.
    We take the starting location, the list of destinations, the graph, the chosen weight - 
    'travel_time' or 'length' - and the two previous lists. 
    This is a recursive function.  While there are destinations still to visit, the reached
    destination becomes the starting point for the next function call. 
    """
   
    if not destinations_nodes:
        return 'Done'    
    # For each destination, calculate the distance to the starting point
    distances = []
    for d in destinations_nodes:
        distances.append(dist(start_node, d))
    # Get the index of the closest destination
    minIndex = distances.index(min(distances))
    # Get the corresponding destination node
    minNode = destinations_nodes[minIndex]
    # Add this to the list of visited destinations
    visited.append(minNode)
    # Get the optimal path to this destination
    path = nx.astar_path(graph, start_node, minNode, weight=weight, heuristic=dist)
    # Add this to the current route
    route.append(path)
    # Remove this destination from the list
    destinations_nodes.pop(minIndex)
    # If there are still destinations to visit, call the function again with the 
    # current location as the new starting point
    if destination_nodes:
        aStarMulti(minNode, destinations_nodes, graph, weight, visited, route)

In [27]:
# In our example we are working with two clusters 
# Get the route for the first cluster of destinations
aStarMulti(start_node, destination_nodes[1], G, 'travel_time', visited, route)

In [28]:
# Check that destinations have been visited
visited

[392166620, 392164099, 392177756]

In [29]:
# Our current route is a list of lists, and since the end point of each leg becomes
# the starting point for the next one, these nodes appear twice. 
# Here we want to remove this duplication while also converting the route into one list
route_list = []
route_list.append(route[0][0])
# For each leg of the route
for i in route:
    # For each node in the leg
    for j in i:
        if route_list[-1] != j:
            route_list.append(j)

In [30]:
def direction_between_points(pt1, pt2):
    """
    Takes two tuples of latitudes and longitudes.
    Returns the compass direction from point 1 to point 2.
    """
    delta_x = pt2[1] - pt1[1]
    delta_y = pt2[0] - pt1[0]
    
    degrees = atan2(delta_x, delta_y) / pi*180
    
    if degrees < 0:
        degrees = 360 + degrees
    
    compass = ['north', 'northeast', 'east', 'southeast', 
               'south', 'southwest', 'west', 'northwest', 'north']
    
    compass_dir = round(degrees/45)
    return compass[compass_dir]

In [31]:
def full_route_info(graph, route, dests_list):
    """
    Creates a DataFrame containing the starting node, street name,
    length, direction of travel, speed, and ending node, for each edge in the graph. 
    We will also label which nodes represent the user's destinations. 
    """
    route_info = list()
    num_steps = len(route)
    i = 0
    j = 1
# We want to get the information for every step in the route
# We handle if any information is unlisted or unknown
    while j < num_steps:
        step_info = list()
        step_info.append(route[i])
        try:
            name = graph[route[i]][route[j]][0]['name']
            # Sometimes a street may change names within a segment - for example, switching
            # from west to east.  In these cases we will just take the first name.
            if type(name) == list:
                name = name[0]
        except KeyError:
            # Sometimes street names are labeled as "links"
            # In these cases I will relabel them with the previous street
            try:
                name = graph[route[i-1]][route[j-1]][0]['name']
            except KeyError:
                name = 'Unnamed'
        step_info.append(name)
        try:
            seg_len = graph[route[i]][route[j]][0]['length']
        except KeyError:
            seg_len = 0
        step_info.append(seg_len)
        # We want to calculate the direction the edge is travelling in
        start = node_to_coords(graph, route[i])
        end = node_to_coords(graph, route[j])
        # Use above function to calculate the direction from the start of the 
        # edge to its end
        step_info.append(direction_between_points(start, end))
        
        seg_speed = graph[route[i]][route[j]][0]['speed_kph']
        step_info.append(seg_speed)
        step_info.append(route[j])
        # Identify if the end node of the edge is a destination
        if route[j] in dests_list:
            step_info.append('Yes')
        else:
            step_info.append('No')
        
        route_info.append(step_info)
        i +=1
        j +=1

    # Create a data frame of this information
    columns = ['Start Node', 'Street Name', 'Length', 'Direction', 'Speed', 'End Node', 'Destination?']
    return pd.DataFrame(route_info, columns=columns)

In [32]:
full_route_df = full_route_info(G, route_list, visited)

In [33]:
def get_turn_dir(graph, node1, node2, node3):
    """
    node1 is starting node for first segment,
    node2 is the shared node between segments,
    node3 is the end node of second segment.
    """
    # First, get coordinates for each of the nodes
    p1 = [graph.nodes[node1]['x'], graph.nodes[node1]['y']]
    p2 = [graph.nodes[node2]['x'], graph.nodes[node2]['y']]
    p3 = [graph.nodes[node3]['x'], graph.nodes[node3]['y']]
    
    # Next, define the line segments as vectors
    p1top2 = [m - n for m,n in zip(p2,p1)]
    p2top3 = [m - n for m,n in zip(p3,p2)]
    p1top3 = [m - n for m,n in zip(p3,p1)]
    
    # Next, compute the cross product of the vectors
    crossprod = np.cross(p1top3, p1top2)
    
    if crossprod > 0:
        return 'Right'
    elif crossprod < 0:
        return 'Left'
    else:
        return 'No turn'

In [34]:
def add_turns(route_df, graph):
    """
    Locate all the rows where the street name changes in the following row.
    Create a dictionary with these as keys - the values will calculate the direction
    of the turn using the previous function. 
    Adds a new column in the df showing the turn direction.
    """
    # Get the rows where the street changes in the following row
    # These will be the turns
    turn_indexes = []
    for ind in route_df.index:
        if ind == route_df.index[-1]:
            pass
        elif route_df.iloc[ind]['Street Name'] == route_df.iloc[ind+1]['Street Name'] and ind != route_df.index[-1]:
            pass
        else:
            turn_indexes.append(ind)
    # Use the above function to get the direction of these turns 
    turns_to_make = dict()
    for ind in turn_indexes:
        turn_dir = get_turn_dir(graph, route_df.iloc[ind]['Start Node'], route_df.iloc[ind]['End Node'], route_df.iloc[ind+1]['End Node'])
        turns_to_make[ind] = turn_dir
    # Create a new column in the dataframe for this information
    turns_list = [None] * len(route_df)
    for i in turns_to_make.keys():
        turns_list[i] = turns_to_make[i]
    
    route_df['Turn Direction'] = turns_list
    return route_df

In [35]:
route_df = add_turns(full_route_df, G)
route_df

,Start Node,Street Name,Length,Direction,Speed,End Node,Destination?,Turn Direction
0,392180675,Dundas Street East,47.679,west,50.0,392173612,No,None
1,392173612,Dundas Street East,56.635,west,50.0,2963661532,No,None
2,2963661532,Dundas Street East,44.804,west,50.0,702874,No,Left
3,702874,Dundas Street West,102.749,west,50.0,394352259,No,None
4,394352259,Dundas Street West,100.070,west,50.0,392167102,No,None
...,...,...,...,...,...,...,...,...
71,392163340,Bridgewater Avenue,82.683,west,46.5,392174994,No,None
72,392174994,Bridgewater Avenue,88.653,west,46.5,392177341,No,Right
73,392177341,Willowbrook Drive,57.301,northwest,46.5,392172769,No,None
74,392172769,Willowbrook Drive,120.762,northwest,46.5,392177431,No,Left


#### Simplifying Route DF

As can be seen in the above output, having multiple destinations along a single route creates two situations that have to be addressed in simplifying the output:
- Once you reach a destination, you will stop.  We want to alert the driver that they have reached a destination.
- From there, the driver will continue along the same street as they are currently on.
- Also, there is the potential for streets that were used in previous legs of the route to be returned to later on.  

In simplifying the route information, we want to group all rows where the driver is just travelling along a single road without a destination or turn.  We sum these lenghts and maintain the starting and ending nodes appropriately.  We include all rows where there is a destination or a turn to another street.

In [36]:
def get_break_points(route_df):
    """
    Get the row numbers where there are turns or destinations. 
    These rows will be kept in the route summary, and all rows in between each
    break point will be condensed into a single entry with the total length and
    appropriate starting and ending nodes.
    """
    break_points = []
    for row in range(len(route_df)):
        if route_df['Destination?'][row]=='Yes' or route_df['Turn Direction'][row] != None:
            break_points.append(row)
        row += 1
    return break_points

In [37]:
break_points = get_break_points(route_df)
print(break_points)

[2, 11, 13, 14, 16, 18, 19, 20, 23, 29, 32, 33, 34, 35, 36, 40, 42, 44, 46, 48, 50, 54, 56, 57, 58, 59, 60, 62, 66, 67, 68, 69, 70, 72, 74, 75]


In [38]:
segments = []
def get_seg(start, segments, break_points, route_df):
    """
    This function will recursively create a list containing lists of the indexes for each leg.
    The identified breakpoints will be the starting points for each sublist.
    
    """
    seg = []
    for row in range(start, len(route_df)):
        seg.append(row)
        if row in break_points:
            break
    segments.append(seg)
    current = seg[-1] + 1
    if current <= len(route_df)-1:
        get_seg(current, segments, break_points, route_df)

In [39]:
segments = []
get_seg(0, segments, break_points, route_df)
print(segments)

[[0, 1, 2], [3, 4, 5, 6, 7, 8, 9, 10, 11], [12, 13], [14], [15, 16], [17, 18], [19], [20], [21, 22, 23], [24, 25, 26, 27, 28, 29], [30, 31, 32], [33], [34], [35], [36], [37, 38, 39, 40], [41, 42], [43, 44], [45, 46], [47, 48], [49, 50], [51, 52, 53, 54], [55, 56], [57], [58], [59], [60], [61, 62], [63, 64, 65, 66], [67], [68], [69], [70], [71, 72], [73, 74], [75]]


In [40]:
def get_simple_route(route_df, segments):
    """
    Produce a dataframe with the simplified route information as described above.
    """
    simple_route = []
    for leg in segments:
        # Segments containing a single index are added to the route
        # These are segments that end in destinations or turns
        if len(leg) == 1:
            leg_info = dict()
            leg_info['Start Node'] = route_df.loc[leg[0]]['Start Node']
            leg_info['Street Name'] = route_df.loc[leg[0]]['Street Name']
            leg_info['Length'] = route_df.loc[leg[0]]['Length']
            leg_info['Direction'] = route_df.loc[leg[0]]['Direction']
            leg_info['Speed'] = route_df.loc[leg[0]]['Speed']
            leg_info['End Node'] = route_df.loc[leg[0]]['End Node']
            leg_info['Destination?'] = route_df.loc[leg[0]]['Destination?']
            leg_info['Turn Direction'] = route_df.loc[leg[0]]['Turn Direction']
            simple_route.append(leg_info)
        else:
            # Longer segments will be combined
            # The end of these segments will be turns or destinations
            leg_info = dict()
            leg_info['Start Node'] = route_df.loc[leg[0]]['Start Node']
            leg_info['Street Name'] = route_df.loc[leg[0]]['Street Name']
            length = 0
            # Combine the lenghts into one value
            for row in leg:
                length += route_df.loc[row]['Length']
            leg_info['Length'] = length
            leg_info['Direction'] = route_df.loc[leg[0]]['Direction']
            leg_info['Speed'] = route_df.loc[leg[0]]['Speed']
            # The end node will correspond to the last leg in this segment
            leg_info['End Node'] = route_df.loc[leg[-1]]['End Node']
            leg_info['Destination?'] = route_df.loc[leg[-1]]['Destination?']
            leg_info['Turn Direction'] = route_df.loc[leg[-1]]['Turn Direction']
            simple_route.append(leg_info)
    return pd.DataFrame(simple_route)

In [41]:
simplified_route = get_simple_route(route_df, segments)
simplified_route

,Start Node,Street Name,Length,Direction,Speed,End Node,Destination?,Turn Direction
0,392180675,Dundas Street East,149.118,west,50.0,702874,No,Left
1,702874,Dundas Street West,817.685,west,50.0,289675303,No,Left
2,289675303,Annes Street,243.056,south,49.6,392176066,No,Right
3,392176066,Dunlop Street West,91.156,west,46.5,392180442,No,Left
4,392180442,Calais Street,920.894,southwest,46.5,392166620,Yes,None
5,392166620,Calais Street,920.894,east,46.5,392180442,No,Right
6,392180442,Dunlop Street West,91.156,east,46.5,392176066,No,Left
7,392176066,Annes Street,231.695,north,50.0,8156250739,No,Right
8,8156250739,Dundas Street West,325.417,east,50.0,392173968,No,Left
9,392173968,Palace Street,923.617,northwest,46.5,392166564,No,Right


In [42]:
def print_route_summary(simplified_route, dest_dict, graph):
    """
    Display a simple summary with all relevant information for the driver.
    """
    # Get the destination nodes
    destinations = simplified_route[simplified_route['Destination?']=='Yes']['End Node']
    num_dest = len(destinations)
    
    print('You have {} locations to vist:'.format(num_dest))
    # Using the dictionary, print off the destination addresses
    for d in destinations:
        for key, value in dest_dict.items():
            if value == d:
                print(key)
    print('\nStart!')
    
    for ind in simplified_route.index:
        # Display information for the leg of the route
        direct = simplified_route['Direction'][ind]
        name = simplified_route['Street Name'][ind]
        length = simplified_route['Length'][ind]
        speed = simplified_route['Speed'][ind]
        print('Head {} on {} for {:.2f} metres at {} km/h.'.format(direct, name, length, speed))
        # At the end of the leg, get the turn 
        turn = simplified_route['Turn Direction'][ind]
        # If there is no turn there are two cases - either it is marked "continue" - 
        # in cases where a road changes speed or its name changes from west to east, for example - 
        # or when we reach the last node on the route
        is_dest = simplified_route['Destination?'][ind]
        if turn == None and is_dest == 'No':
            # If we aren't at a destination, write continue
            if ind != simplified_route.index[-1]:
                print('Continue')
            else:
                # Display no turn information when the route is finished 
                pass
        elif turn == None and is_dest == 'Yes':
            # Have arrived at a destination.
            destnode = simplified_route['End Node'][ind]
            # Get the address 
            for key, value in dest_dict.items():
                if value == destnode:
                    add = key
            print('\nYou have reached a destination:', add)
            # This destination is unlikely to be at an actual graph node, so we want to provide some info
            # Get the coordinates of the node and the actual address
            nodecoords = node_to_coords(graph, destnode)
            addcoords = address_to_coord(add)
            # Calculate the distance between these
            dest_dist = geopy.distance.distance(nodecoords, addcoords).m
            # Calculate the direction in which the destination will be
            dest_dir = direction_between_points(nodecoords, addcoords)
            print('It is {:.2f} metres to your {}.\n'.format(dest_dist, dest_dir))
        else:
            # If there is a turn, indicate it
            print('Turn', turn.upper())
    
    print('You have visited all of your destinations!')

In [43]:
print_route_summary(simplified_route, dest_dict, G)

You have 3 locations to vist:
910 Dundas St W Whitby
3100 Garden Street Whitby
3500 Brock St N Whitby

Start!
Head west on Dundas Street East for 149.12 metres at 50.0 km/h.
Turn LEFT
Head west on Dundas Street West for 817.69 metres at 50.0 km/h.
Turn LEFT
Head south on Annes Street for 243.06 metres at 49.6 km/h.
Turn RIGHT
Head west on Dunlop Street West for 91.16 metres at 46.5 km/h.
Turn LEFT
Head southwest on Calais Street for 920.89 metres at 46.5 km/h.

You have reached a destination: 910 Dundas St W Whitby
It is 847.18 metres to your west.

Head east on Calais Street for 920.89 metres at 46.5 km/h.
Turn RIGHT
Head east on Dunlop Street West for 91.16 metres at 46.5 km/h.
Turn LEFT
Head north on Annes Street for 231.69 metres at 50.0 km/h.
Turn RIGHT
Head east on Dundas Street West for 325.42 metres at 50.0 km/h.
Turn LEFT
Head northwest on Palace Street for 923.62 metres at 46.5 km/h.
Turn RIGHT
Head east on Beech Street West for 301.50 metres at 46.5 km/h.
Turn RIGHT
Head eas

#### Customizations

In [44]:
def avoid_a_street(graph, street_list):
    """
    Will take a list of street names and return a new graph with these
    edges removed.
    Since the graph is defined at the start and will be used for all uses,
    we first make a copy of the graph before manipulating it.
    Returns a new graph object with appropriate edges removed.
    """
    G2 = nx.Graph.copy(graph)
    edges_to_remove = []
    for edge in G2.edges:
        try:
            # Get the street name for each edge segment
            seg = G2.edges[edge][0]['name']
            # Add to list of edges to be removed
            if seg in street_list:
                edges_to_remove.append(edge)
        except KeyError:
            pass
    # Remove edges
    G2.remove_edges_from(edges_to_remove)
    return G2

In [45]:
def avoid_a_roadtype(graph, street_type_list):
    """
    Will take a road type - 'motorway', 'secondary', 'tertiary', 'residential' - 
    and remove this edges from the graph. 
    This will allow a user to avoid high speed, or low speed, roads, if possible.
    
    NOTE:
    This function will always follow avoid_a_street and so will take a copy of a graph.
    Graph copies have their information stored at inside a nested dictionary.
    """
    G2 = nx.Graph.copy(graph)
    edges_to_remove = []
    for edge in G2.edges:
        try:
            # The highway attribute of the edge gives the road type
            seg = G2.edges[edge][0][0]['highway']
            if seg in street_type_list:
                edges_to_remove.append(edge)
        except KeyError:
            pass
    G2.remove_edges_from(edges_to_remove)
    return G2

### Getting User Specifications

In [46]:
def get_starting_loc():
    """
    Prompts the user for their starting address.
    """
    print('Enter your starting address and city in full.')
    print('Ex: 23 Vanier Street Whitby')
    start = input()
    return start.title()

In [47]:
def get_destinations():
    """
    Prompts the user for their destinations.
    """
    print('\nEnter your destination addresses and cities in full.')
    print('Press enter after each entry.')
    print('Type "DONE" when you are finished.')
    destinations = []
    while True:
        dest = input()
        if dest.upper() == 'DONE':
            break
        elif dest:
            destinations.append(dest.title())
    return destinations

In [48]:
def get_customizations():
    """
    Asks the user to enter a list of streets they want to avoid,
    one at a time.
    Returns a list of street names.
    """
    print('\nEnter any streets you want to avoid.')
    print('Enter in full. ex: "Brock Street".')
    print('Press enter after each entry.')
    print('Type "DONE" when you are finished.')
    avoid = []
    while True:
        street = input()
        if street.upper() == 'DONE':
            break
        else:
            avoid.append(street.title())
    return avoid

In [49]:
def get_customizations2():
    """
    Gets further user specifications based on road speeds.
    Returns a list of road types to be removed.
    """
    avoid = []
    highways = input('Avoid highways? (Y/N): ')
    if highways.lower().startswith('y'):
        avoid.append('motorway')

    return avoid

In [50]:
def get_customizations3():
    """
    Gets further user specifications on what to optimize on.
    """
    how_opt = input('Do you want to optimize based on travel time (T) or distance traveled (D)? ')
    if how_opt.lower().startswith('T'):
        return 'travel_time'
    else:
        return 'length'

### Putting it Together

In [51]:
# Build the graph - this is Whitby/Oshawa for testing purposes - east, west, south, north
G = build_graph(43.984503, 43.862879, -78.954552, -78.821660)

In [52]:
def run_multi_opt_path():
    """
    This is the main function that takes all user inputs and returns the optimized routes 
    for each of the drivers. 
    """
    # Get the user's starting location and destination
    # If their input does not provide coordinates - if they enter a spelling of a street
    # that does not exist - the address_to_coord function will alert them and here they will
    # be reprompted for a valud address. 
    while True:
        start = get_starting_loc()
        start = address_to_coord(start)
        if start:
            break
    
    # Get destination addresses 
    dest = get_destinations()
    # Get the coordinates of each
    dest_coords = []
    for i in dest:
        dest_coords.append(address_to_coord(i))
    
    # Store the destination addresses and corresponding graph nodes
    dest_dict = dest_address_and_node(G, dest)
    
    # Prompt user for the number of drivers    
    num_drivers = int(input('How many drivers are available? (Ex: 2) '))
    
    # Cluster the destinations according to this number
    clusters = cluster_destinations(dest_coords, num_drivers)
    
    # Get the user's customizations 
    roads_to_avoid = get_customizations()
    speeds_to_avoid = get_customizations2()
    opt_type = get_customizations3()
    
    # Use the functions to remove necessary edges from the graph
    G2 = avoid_a_street(G, roads_to_avoid)
    G3 = avoid_a_roadtype(G2, speeds_to_avoid)
    
    # Convert locations into nodes
    start_node = coords_to_node(G2, start)
    # Get destination nodes
    destination_nodes = []
    for i in range(len(clusters)):
        inodes = []
        for x in clusters[i]:
            inodes.append(coords_to_node(G, x))
        destination_nodes.append(inodes)
    
    # For each driver, get their route
    for driver in range(len(clusters)):
        # Display the driver number
        print('\n\nDriver', driver+1, 'Route:')
        # Store their destination nodes 
        your_dests = destination_nodes[driver]
        # Create empty list of their visited destinations and their overall route
        visited = []
        route = []
        # Use the algorithm to build these lists
        aStarMulti(start_node, your_dests, G3, opt_type, visited, route)
        
        # Convert the list of lists into a single list
        route_list = []
        route_list.append(route[0][0])
        for i in route:
            for j in i:
                if route_list[-1] != j:
                    route_list.append(j)
        
        # Check if there is a route
        if len(route_list) == 1:
            print('There is no possible route with these specification.')
            print('Please start over.')
            break

        # Get the full route information
        full_route_df = full_route_info(G, route_list, visited)
        # Add in the turns
        route_df = add_turns(full_route_df, G)
        # Get the breakpoints in this driver's route
        break_points = get_break_points(route_df)
        # Split their route into segments
        segments = []
        get_seg(0, segments, break_points, route_df)
        # Simplify the route
        simplified_route = get_simple_route(route_df, segments)
        # Print the driver's route
        print_route_summary(simplified_route, dest_dict, G)

### TESTING

SCENARIO:  Canada Post has a depot in Whitby, along with six post office locations.  We have a certain number of trucks that will take sorted mail from the depot to the various locations for delivery in their individual areas.

![Post Offices Whitby](canada_post.png)

Start:  209 Dundas St E Whitby

Destinations:  

dest1 = 910 Dundas St W Whitby

dest2 = 3100 Garden Street Whitby

dest3 = 4081 Thickson Road N Whitby

dest4 = 728 Anderson Street Whitby

dest5 = 1801 Dundas St E Whitby

dest6 = 3500 Brock St N Whitby

In [55]:
run_multi_opt_path()

Enter your starting address and city in full.
Ex: 23 Vanier Street Whitby
209 Dundas St E Whitby

Enter your destination addresses and cities in full.
Press enter after each entry.
Type "DONE" when you are finished.
910 Dundas St W Whitby
3100 Garden Street Whitby
4081 Thickson Road N Whitby
done
How many drivers are available? (Ex: 2) 1

Enter any streets you want to avoid.
Enter in full. ex: "Brock Street".
Press enter after each entry.
Type "DONE" when you are finished.
Rossland Road East
Rossland Road West
Taunton Road East
done
Avoid highways? (Y/N): y
Do you want to optimize based on travel time (T) or distance traveled (D)? t


Driver 1 Route:
You have 3 locations to vist:
910 Dundas St W Whitby
3100 Garden Street Whitby
4081 Thickson Road N Whitby

Start!
Head west on Dundas Street East for 149.12 metres at 50.0 km/h.
Turn LEFT
Head west on Dundas Street West for 817.69 metres at 50.0 km/h.
Turn LEFT
Head south on Annes Street for 243.06 metres at 49.6 km/h.
Turn RIGHT
Head wes

In [ ]:
# Create a situation where no path will be possible

In [322]:
run_multi_opt_path()

Enter your starting address and city in full.
Ex: 23 Vanier Street Whitby
209 dundas street east

Enter your destination addresses and cities in full.
Press enter after each entry.
Type "DONE" when you are finished.
910 dundas street west
done
How many drivers are available? (Ex: 2) 1

Enter any streets you want to avoid.
Enter in full. ex: "Brock Street".
Press enter after each entry.
Type "DONE" when you are finished.
dundas street east
dundas street west
perry street
done
Avoid highways? (Y/N): n
Do you want to optimize based on travel time (T) or distance traveled (D)? d


Driver 1 Route:
There is no possible route with these specification.
Please start over.


In [323]:
run_multi_opt_path()

Enter your starting address and city in full.
Ex: 23 Vanier Street Whitby
23 vanier street whitby

Enter your destination addresses and cities in full.
Press enter after each entry.
Type "DONE" when you are finished.
11 teagarden court whitby
done
How many drivers are available? (Ex: 2) 1

Enter any streets you want to avoid.
Enter in full. ex: "Brock Street".
Press enter after each entry.
Type "DONE" when you are finished.
done
Avoid highways? (Y/N): n
Do you want to optimize based on travel time (T) or distance traveled (D)? t


Driver 1 Route:
You have 1 locations to vist:
11 Teagarden Court Whitby

Start!
Head southeast on Vanier Street for 233.68 metres at 46.5 km/h.
Turn LEFT
Head east on Kenneth Hobbs Avenue for 600.90 metres at 46.5 km/h.
Turn RIGHT
Head east on Bassett Boulevard for 185.05 metres at 46.5 km/h.
Turn RIGHT
Head east on Lumsden Crescent for 450.28 metres at 46.5 km/h.
Turn RIGHT
Head southeast on Bassett Boulevard for 242.00 metres at 46.5 km/h.
Turn LEFT
Head ea